In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras import metrics


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
dftrain = pd.read_csv(train_file_path,sep="\t",header=None)
dftrain.columns = ['label', 'text']
dftrain

,label,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [ ]:
dftest = pd.read_csv(test_file_path,sep="\t",header=None)
dftest.columns = ['label', 'text']
dftest

,label,text
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [ ]:
data = dftrain.append(dftest, ignore_index=True)
data

<ipython-input-6-52ddb49041ed>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = dftrain.append(dftest, ignore_index=True)


,label,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
5566,ham,true dear..i sat to pray evening and felt so.s...
5567,ham,"what will we do in the shower, baby?"
5568,ham,where are you ? what are you doing ? are yuou ...
5569,spam,ur cash-balance is currently 500 pounds - to m...


In [ ]:
tags = data["label"]
texts = data["text"]
num_max = 1000
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)

mat_data = tok.texts_to_matrix(texts,mode='count')
mat_train = tok.texts_to_matrix(dftrain,mode='count')
mat_test = tok.texts_to_matrix(dftest,mode='count')

max_len = 100
x_train = tok.texts_to_sequences(dftrain["text"])
x_test = tok.texts_to_sequences(dftest["text"])
train = sequence.pad_sequences(x_train,maxlen=max_len)
max_len = 100
test = sequence.pad_sequences(x_test,maxlen=max_len)

labeltrain = dftrain["label"]
traintags = le.fit_transform(labeltrain)
labeltest = dftest["label"]
testtags = le.fit_transform(labeltest)

In [ ]:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2
   
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(train,traintags,batch_size=32,epochs=10,verbose=1,validation_split=0.3)


Epoch 1/10
92/92 [==============================] - 43s 395ms/step - loss: 8760.3320 - acc: 0.8708 - binary_accuracy: 0.8708 - val_loss: 0.4888 - val_acc: 0.8517 - val_binary_accuracy: 0.8517
Epoch 2/10
92/92 [==============================] - 36s 391ms/step - loss: 0.4632 - acc: 0.9207 - binary_accuracy: 0.9207 - val_loss: 0.3062 - val_acc: 0.9553 - val_binary_accuracy: 0.9553
Epoch 3/10
92/92 [==============================] - 40s 432ms/step - loss: 307167199232.0000 - acc: 0.9593 - binary_accuracy: 0.9593 - val_loss: 193.6011 - val_acc: 0.9171 - val_binary_accuracy: 0.9171
Epoch 4/10
92/92 [==============================] - 36s 397ms/step - loss: 8.5917 - acc: 0.9703 - binary_accuracy: 0.9703 - val_loss: 32.2852 - val_acc: 0.9593 - val_binary_accuracy: 0.9593
Epoch 5/10
92/92 [==============================] - 37s 400ms/step - loss: 1.5059 - acc: 0.9774 - binary_accuracy: 0.9774 - val_loss: 41.1959 - val_acc: 0.9601 - val_binary_accuracy: 0.9601
Epoch 6/10
92/92 [===================

In [ ]:
_,accuracy,_ =model.evaluate(test,testtags)
accuracy

44/44 [==============================] - 3s 70ms/step - loss: 0.8988 - acc: 0.9734 - binary_accuracy: 0.9734


0.9734195470809937

In [ ]:
string = ["What you doing?how are you?"]
inputstr = tok.texts_to_sequences(string)
pred = model.predict(inputstr)
output = [pred[0][0], 'ham'] 
output

1/1 [==============================] - 1s 918ms/step


[0.10914365, 'ham']

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  inputstr = tok.texts_to_sequences([pred_text])
  pred = model.predict(inputstr)[0][0]

  if pred >= 0.5:
    label = "spam"
  else:
    label = "ham"

  output = [pred, label] 

  return output

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 333ms/step
[0.115825765, 'ham']


In [ ]:
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 52ms/step
You passed the challenge. Great job!
